# Do the mappings to deepwalk ids

In [1]:
import pandas as pd

In [2]:
nodemap = (pd
    .read_csv("data/node_map.tsv", sep='\t')
    [["node_uid", "node_id"]]
)

In [3]:
def format_data(fname):
    return (pd
        .read_csv(fname, sep='\t')
        [["chemical_id", "disease_id", "etype"]]
        .fillna(0)
        .replace("TREATS_CDtDO", 1)
    )

In [4]:
def add_uid(df):
    return (df
        .merge(
            nodemap, how="inner", left_on="chemical_id",
            right_on="node_id"
        )
        .drop("node_id", axis=1)
        .rename(columns={"node_uid": "chemical_uid"})
        .merge(
            nodemap, how="inner", left_on="disease_id",
            right_on="node_id"
        )
        .drop("node_id", axis=1)
        .rename(columns={"node_uid": "disease_uid"})
    )

In [5]:
K = 1
for i in range(K):
    train = (
        format_data("data/training/training_piece{}.tsv".format(i))
        .pipe(add_uid)
    )
    
    holdout = (
        format_data("data/holdout/holdout_piece{}.tsv".format(i))
        .pipe(add_uid)
    )
    
    train.to_csv(
        "data/training/clean_train{}.tsv".format(i),
        sep='\t', index=False
    )
    
    holdout.to_csv(
        "data/holdout/clean_holdout{}.tsv".format(i),
        sep='\t', index=False
    )